## Understanding the internals of NumPy to avoid unnecessary array copying
We can achieve significant performance speed enhancement with NumPy over native Python code, particularly when our computations follow the **Single Instruction, Multiple Data (SIMD)** paradigm.
<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/c/ce/SIMD2.svg/1920px-SIMD2.svg.png' width=300 align='left'>

It describes computers with multiple processing elements that perform the same operation on multiple data points simultaneously. Such machines exploit data level parallelism.

In [1]:
import numpy as np
def aid(x):
    # This function returns the memory
    # block address of an array.
    return x.__array_interface__['data'][0]

In [2]:
a = np.zeros(3)
aid(a), aid(a[1:])

(140627578758800, 140627578758808)

A general and reliable solution to find out whether two arrays share the same data:

In [3]:
def get_data_base(arr):
    """For a given Numpy array, find the base array
    that owns the actual data."""
    base = arr
    while isinstance(base.base, np.ndarray):
        base = base.base
    return base

def arrays_share_data(x, y):
    return get_data_base(x) is get_data_base(y)

In [4]:
print(arrays_share_data(a, a.copy()))

False


In [5]:
print(arrays_share_data(a, a[:1]))

True


### How to do it
Computations with NumPy arrays may involve copies between blocks of memory. These copies are not always necessary, in which case they should be avoided.

1. May need to make a copy of an array; e.g. if we need to manipulate an array while keeping an original copy in memory:

In [6]:
import numpy as np
a = np.zeros(10)
ax = aid(a)
ax

140627609725040

In [7]:
b = a.copy()
aid(b) == ax

False

2. Array computations can involve in-place operations (the first example in the following code: the array is modified) or implicit-copy operations (the second example: a new array is created):

In [9]:
a *= 2
aid(a) == ax

True

In [10]:
c = a*2
aid(c) == ax

False

In [21]:
# Explicity copy operations are slower
%timeit a = np.zeros(1000000); a *= 2

1.27 ms ± 19.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [23]:
%timeit a = np.zeros(1000000); b = a * 2

2.42 ms ± 52.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


3. Reshaping an array may or may not involve a copy. For instance, reshaping a 2D matrix does not involve a copy, unless it is transposed.

In [19]:
a = np.zeros((100, 100))
ax = aid(a)

In [20]:
b = a.reshape((1, -1))
aid(b) == ax

True

In [24]:
c = a.T.reshape((1, -1))
aid(c) == ax

False

In [25]:
# So the latter is significantly slower than the former
%timeit b = a.reshape((1, -1))

313 ns ± 3.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [26]:
%timeit a.T.reshape((1, -1))

7.79 µs ± 45.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


4. Both the `flatten()` and `ravel()` methods of an array reshape it into a 1D vector (a flattened array). However, the `flatten()` method always returns a copy, and the `ravel()` method returns a copy only if necessary (thus faster, especially with large arrays)

In [27]:
d = a.flatten()
aid(d) == ax

False

In [28]:
e = a.ravel()
aid(e) == ax

True

In [29]:
%timeit a.flatten()

2.76 µs ± 43 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
%timeit a.ravel()

187 ns ± 0.762 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


5. **Broadcasting rules** allow us to make computations on arrays with different but compatible shapes. In other words, we don't always need to reshape or tile our arrays to make their shapes match. The following example illustrates two ways of doing an **outer product** between two vectors: the first method involves array tiling, the second one (faster) involves broadcasting.

In [31]:
n = 1000
a = np.arange(n)
ac = a[:, np.newaxis] # column vector
ar = a[np.newaxis, :] # row vector

In [32]:
%timeit np.tile(ac, (1, n)) * np.tile(ar, (n, 1))

6.43 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%timeit ar * ac

1.24 ms ± 7.86 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## How it works
### Why are NumPy arrays efficient?
A Numpy array is basically described by metadata (notably the number of dimensions, the shape, and the data type) and the actual data.

The data is stored in a **homogeneous** and **continguous** block of memory, at a particular address in system memory, at a particular address in system memory (**Random Access Memory (RAM)**). This block of memory is called the **data buffer**.

This is the main difference between an array and a pure Python structure, such as a list, where the items are scattered across the system memory. This is the critical feature that makes NumPy arrays so efficient.

Why is this so important? Main reasons:

- Computations on arrays can be written very efficiently in a low language such as C (a large part of NumPy is actually written in C). Knowing the address of the memory block and the data type, it is just simple arithmetic to loop over all items, for examples. There would be a significant overhead if we did that in Python with a list.
- **Spatial locality** in memory access patterns results in performance gains notably due to the CPU cache. Indeed, the cache loads bytes in chunks from the RAM to the CPU registers. Adjacent items are then loaded very efficiently (**sequential locality**, or **locality of reference**)
- Finally, the fact that items are stored contiguously in memory allows NumPy and to take advantage of **vectorized instructions** of model CPUs, such as Intel's SSE and AVX, AMD's XOP and so on. E.g. multiple consecutive floating point numbers can be loaded in 128-, 256- or 512 bit registers for vectorized arithmetical computations implemented as CPU instructions.

Additionally, NumPy can be linked to highly optimized linear algebra libraries such as `BLAS` and `LAPACK` through **ATLAS** or the **Intel Math Kernel Library (MKL)**. A few specific matrix computations may also be multithreaded, taking advantage of the power of modern multicore processors. 

In conclusion, storing data in a contiguous block of memory ensures that the architecture of modern CPUs is used optimally, in terms of memory access patterns, CPU cache, and vectorized instructions.

### What is the difference between in-place and implicit-copy operations?
An expression such as `a *= 2` corresponds to an in-place operation, where all values of the array are multiplied by two. By contrast, `a = a*2` means that a new array containing the values of `a*2` is created, and the variable `a` now points to this new array. The old array becomes unreferenced and will be deleted by the garbage collector. No memory allocation happens in the first case, unlike the second case.

More generally, expressions such as `a[i:j]` are views to parts of an array; the point to the memory buffer containing the data. Modifying them with inplace operations changes the original array.

Knowing the subtlety of NumPy can help fix bugs and optimize the speed and memory consumption of your code by reducing the number of unnecessary copies.

### Why can't some arrays be reshaped without a copy?
We explain the example in step 3 here, where a transposed 2D matrix cannot be flattened without a copy. A 2D matrix contains items indexed by two numbers (row and column), but it is stored internally as a 1D contiguous block of memory: 
- **row-major order**: We can put the elements of the first row first, then the second row, and so on
- **column-major order**: put the elements of the first column first, then second column...
NumPy uses row-major order, like C, but unlike FORTRAN.

<img src='numpy_order.png' width=300>

More generally, NumPy uses the notion of **strides** to convert between a multidimensional index and the memory location of the underlying (1D) sequence of elements. The specific mapping between `array[i1, i2]` and the relevant byte address of the internal data is given by:

```
offset = array.strides[0] * i1 + array.strides[1] * i2
```

When reshaping an array, NumPy avoids copies when possible by modifying the **strides** attribute. For example, when transposing a matrix, the order of strides is reversed, but the underlying data remains identical. However, flattening a transposed array cannot be simply accomplished by modifying strides, so a copy is needed.

Internal array layout can also explain unexpected performance discrepancies.

In [34]:
a = np.random.rand(5000, 5000)
%timeit a[0, :].sum()
%timeit a[:, 0].sum()

3.68 µs ± 75.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
34.8 µs ± 603 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [37]:
%timeit a.T[0, :].sum()

34.8 µs ± 476 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
